In [23]:
import sys
import os
from IPython.core.interactiveshell import InteractiveShell
from platform import python_version
print(python_version())
InteractiveShell.ast_node_interactivity = "all"
print("My working directory:\n" + os.getcwd())

3.6.8
My working directory:
/home/dsxuser/work


In [24]:
#!pip install geopy

In [25]:
import numpy as np
import pandas as pd

print('Libraries imported.')

Libraries imported.


### Step 0: Read table from Wikipedia

In [26]:
temp = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [27]:
temp[0].head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


<h3>Step 1:</h3>

<ol>
    <li>Take the table we need from result</li>
</ol>

<hr>

In [28]:
df_PostCodeM = pd.DataFrame(temp[0])
#df_PostCodeM.columns = df_PostCodeM.iloc[0]
#df_PostCodeM.drop([0], inplace = True)
#df_PostCodeM.reset_index(drop=True, inplace=True)
df_PostCodeM.head(10)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


<h3>Step 2: drop rows with 'Not assigned' Borough</h3>

In [29]:
df_PostCodeM_adj = df_PostCodeM[df_PostCodeM['Borough'] != 'Not assigned']
df_PostCodeM_adj.reset_index(drop=True, inplace=True)
df_PostCodeM_adj.head(10)
df_PostCodeM_adj.shape

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Not assigned
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


(210, 3)

<h3>Step 3: Replace the 'Not assigned' Neighborhood with the value of borough</h3>

In [30]:
for i in range(df_PostCodeM_adj.shape[0]):
    if df_PostCodeM_adj.iloc[i]['Neighborhood'] == 'Not assigned':
        df_PostCodeM_adj.iloc[i]['Neighborhood'] = df_PostCodeM_adj.iloc[i]['Borough']

In [31]:
(df_PostCodeM_adj['Neighborhood']=='Not assigned').unique()
df_PostCodeM_adj.head(10)

array([False])

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


<h3>Step 4: merge neighborhoods with the same postal code</h3>

In [32]:
#a function to create a dataframe with merged neighborhoods with the same postal code
def merge_neighbor(temp):
    a = str()
    for i in range(temp.shape[0]):
        a = a + temp.iloc[i]['Neighborhood'] +', '
    result_list = [temp.iloc[0,0], temp.iloc[0,1], a[0:-2]]
    result_df = pd.DataFrame(result_list, index=['Postcode', 'Borough', 'Neighborhood']).T
    return result_df

In [33]:
#get all the postal code (no repetition)
Postcode = df_PostCodeM_adj['Postcode'].unique()

#merge neighborhoods for each postal code
for item in Postcode:
    temp = df_PostCodeM_adj[df_PostCodeM_adj['Postcode'] == item]
    #temp
    result_df = merge_neighbor(temp)
    df_PostCodeM_adj = df_PostCodeM_adj[df_PostCodeM_adj['Postcode'] != item]
    df_PostCodeM_adj = df_PostCodeM_adj.append(result_df, ignore_index=True, sort=False)
    #df_PostCodeM_adj.shape

In [34]:
#show 11 rows and shape
df_PostCodeM_adj.head(11)
df_PostCodeM_adj.shape

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


(103, 3)